In [82]:
import pandas as pd
import numpy as np
import json
import ollama

# Load Configuration
def load_config(config_path):
    with open(config_path, "r") as file:
        return json.load(file)

# Create Date Column using last day of the month
def create_date_column(df):
    df["DATE"] = pd.to_datetime(df["YEAR"].astype(str) + "-" + df["MONTH"].astype(str) + "-01") + pd.offsets.MonthEnd(0)
    return df

# Load and Preprocess Data
def preprocess_data(data_path, kpi_column, groupby_columns=None):
    df = pd.read_csv(data_path)
    
    if groupby_columns:
        group_columns = ["YEAR", "MONTH"] + groupby_columns
    else:
        group_columns = ["YEAR", "MONTH"]
    
    df = df.groupby(group_columns).agg({kpi_column: "sum"}).reset_index()
    df = create_date_column(df)
    df.sort_values("DATE", inplace=True)
    return df

# Descriptive Analysis
def descriptive_analysis(df, kpi_column, config, groupby_columns=None):
    threshold = config["kpi_threshold"]
    ratechange_threshold = config["ratechange_threshold"]
    
    df["Above_Threshold"] = df[kpi_column] > threshold
    df["Rate_Change"] = df[kpi_column].pct_change()
    df["Rate_Above_Threshold"] = df["Rate_Change"] > ratechange_threshold
    
    latest_above_threshold = df[df["Above_Threshold"]].sort_values("DATE", ascending=False).iloc[0]
    
    summary = {
        "latest_period_above_threshold": {
            "month": latest_above_threshold["MONTH"],
            "year": latest_above_threshold["YEAR"],
            "value": latest_above_threshold[kpi_column]
        },
        "current_vs_last_year": df[kpi_column].pct_change(12).iloc[-1],
        "current_vs_last_month": df[kpi_column].pct_change().iloc[-1],
    }
    
    if groupby_columns:
        top_values = df[df["Above_Threshold"]][["MONTH"] + groupby_columns + [kpi_column] + ["DATE"]]
        top_values_sorted = top_values.sort_values(by=["DATE", kpi_column], ascending=[False, False]).head(10)
        summary["top_10_values_above_threshold"] = top_values_sorted.to_dict(orient="records")
        
        top_rate_changes = df[df["Rate_Above_Threshold"]][["MONTH"] + groupby_columns + ["Rate_Change"] + ["DATE"]]
        top_rate_changes_sorted = top_rate_changes.sort_values(by=["DATE", "Rate_Change"], ascending=[False, False]).head(10)
        summary["top_10_ratechange_above_threshold"] = top_rate_changes_sorted.to_dict(orient="records")
        
        min_row = df.loc[df[kpi_column].idxmin()]
        max_row = df.loc[df[kpi_column].idxmax()]
        summary["min_value"] = {"value": min_row[kpi_column], "month": min_row["MONTH"], "group": {col: min_row[col] for col in groupby_columns}}
        summary["max_value"] = {"value": max_row[kpi_column], "month": max_row["MONTH"], "group": {col: max_row[col] for col in groupby_columns}}
    else:
        summary["min_value"] = df[kpi_column].min()
        summary["max_value"] = df[kpi_column].max()
    
    return summary



In [83]:
def main(groupby_columns=None):
    config = load_config("config.json")
    df = preprocess_data("Warehouse_and_Retail_Sales_enhanced.csv", "RETAIL SALES", groupby_columns=groupby_columns)
    result = descriptive_analysis(df, "RETAIL SALES", config, groupby_columns=groupby_columns)
    return result

In [84]:
config = load_config("config.json")


In [85]:
config

{'kpi_threshold': 11668.35,
 'ratechange_threshold': 0.1,
 'time_period': '12 months',
 'groupby_columns': ['SUPPLIER']}

In [86]:
df = preprocess_data("Warehouse_and_Retail_Sales_enhanced.csv", "RETAIL SALES", groupby_columns=["COUNTRY"])


In [87]:
df.head()

,YEAR,MONTH,COUNTRY,RETAIL SALES,DATE
0,2017,6,Australia,11668.35,2017-06-30
1,2017,6,Canada,14606.15,2017-06-30
2,2017,6,France,10369.96,2017-06-30
3,2017,6,Germany,12020.38,2017-06-30
4,2017,6,Italy,10339.10,2017-06-30


In [88]:
result = descriptive_analysis(df, "RETAIL SALES", config, groupby_columns=["COUNTRY"])


In [89]:
groupby_columns = ["COUNTRY"]
main(groupby_columns)

{'latest_period_above_threshold': {'month': 7,
  'year': 2020,
  'value': 12842.94},
 'current_vs_last_year': -0.08666522772102525,
 'current_vs_last_month': 0.18326672490651696,
 'top_10_values_above_threshold': [{'MONTH': 7,
   'COUNTRY': 'UK',
   'RETAIL SALES': 13552.92,
   'DATE': Timestamp('2020-07-31 00:00:00')},
  {'MONTH': 7,
   'COUNTRY': 'USA',
   'RETAIL SALES': 13485.74,
   'DATE': Timestamp('2020-07-31 00:00:00')},
  {'MONTH': 7,
   'COUNTRY': 'Germany',
   'RETAIL SALES': 12842.94,
   'DATE': Timestamp('2020-07-31 00:00:00')},
  {'MONTH': 7,
   'COUNTRY': 'Italy',
   'RETAIL SALES': 12694.48,
   'DATE': Timestamp('2020-07-31 00:00:00')},
  {'MONTH': 3,
   'COUNTRY': 'Italy',
   'RETAIL SALES': 15194.01,
   'DATE': Timestamp('2020-03-31 00:00:00')},
  {'MONTH': 3,
   'COUNTRY': 'USA',
   'RETAIL SALES': 15073.68,
   'DATE': Timestamp('2020-03-31 00:00:00')},
  {'MONTH': 3,
   'COUNTRY': 'Germany',
   'RETAIL SALES': 13940.82,
   'DATE': Timestamp('2020-03-31 00:00:00')},


In [92]:
# Generate Explanation using LLM
def generate_explanation(result):
    prompt = f"Analyze the following KPI results and provide a business explanation with inferences: {result}"
    response = ollama.chat(model="llama3.2:1b", messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"]

In [94]:
response = generate_explanation(result)
print(response)

Analyzing the provided Key Performance Indicator (KPI) results can provide valuable insights into business performance and potential areas for improvement.

**Current Situation:**

The KPIs indicate a slight decline in retail sales from the previous year, with a decrease of -0.08666522772102525 compared to 2020. This suggests that the company's sales have been impacted by various factors, including market conditions, competition, and seasonal fluctuations.

**Breakdown of Key Performance Indicators:**

1. **Latest Period Above Threshold:** The KPI shows that the current period (July 2023) is above the threshold for retail sales in July 2020 by a significant margin (-28% increase). This indicates that the company's sales have outperformed expectations.

2. **Current vs Last Year:** The comparison between the current and last year reveals a slight decline, with a -0.08666522772102525 percentage point difference. This suggests that the company's sales have been impacted by market conditio